In [1]:
import pathlib
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

# importing FinQuant's function to automatically build the portfolio
from finquant.portfolio import build_portfolio
import finquant.quants

In [2]:
## function to read the data from csv file and create / return a dataframe 
def get_data_frame_from_csv(csv_file):
    csv_file_path = pathlib.Path(csv_file)
    data = pd.read_csv(csv_file_path)
    data = clean_data(data)
    #data = data.set_index(pd.to_datetime(data["Date"], infer_datetime_format='%Y-%m-%d'))
    data.head()
    return data

#function to read csv and returns contents as list 
def read_csv(csvFile):
    csvList = []
    with csvFile.open() as f:
        csvReader = csv.reader(f, delimiter=',', quotechar='"')
        counter = 0
        for row in csvReader:
            if (counter==0):
                next(csvReader)
            csvList = list(csvReader)
    return csvList

## function to write the output to file 
def writeOutputToFile(fileName, data):
    File_object = open(fileName, "w")
    File_object.writelines(data)
    File_object.close()



In [3]:
## function to clean the data
def clean_data(data):
    ## count null data
    data.isnull().count()
    ## drop nulls
    data = data.dropna()
    return data

## function to fix Data Types
def fix_data_types(df_slice,to_data_type):
    if (to_data_type=='date'):
        df_slice = pd.to_datetime(df_slice)
        ## df['Date'] = df['Date'].astype('datetime64[ns]')
    elif (to_data_type=='numeric'):
        ## remove the '$',',' symbol in price data 
        df_slice = df_slice.replace({'\$':'','\,':''}, regex = True)
        df_slice = pd.to_numeric(df_slice, downcast="float")
    else:
        return df_slice
    return df_slice



In [4]:
csvFile = pathlib.Path('./Resources/g_and_g_etfs.csv')
#csv_data = read_csv(csvFile)
csv_data = get_data_frame_from_csv('./Resources/g_and_g_etfs.csv')
csv_data = csv_data#.head(16)#.groupby('Type')
#csv_data

In [5]:
def modify_csv_data_with_map_values(type, csv_data):    
    ## TO DO global rather than passing as argument to function 
    ## global csv_data
    csv_data_by_type = csv_data[csv_data.Type==type]
    csv_data_by_type = csv_data_by_type#.head(5)

    tickers = list(csv_data_by_type['Ticker'])     ## ['BLOK', 'BLCN', 'LEGR', 'BITQ', 'DAPP', 'BTTC', 'KOIN', 'DFE', 'DLS', 'DGRW']
    #print(tickers)

    each_allocation = ( 100/len(tickers) )
    allocations = []
    for x in tickers:
        allocations.append(each_allocation)

    # initialise data of lists.
    data = {'Name': tickers,
            'Allocation': allocations}

    # Creates pandas DataFrame.
    df = pd.DataFrame(data)

    # print the data
    #df.index

    pf_allocation = df #pd.DataFrame.from_dict(df, orient="index")
    names = pf_allocation["Name"].values.tolist()
    names

    # dates can be set as datetime or string, as shown below:
    start_date = datetime.datetime(2018, 7, 5)

    pf = build_portfolio(
        names=names, data_api="yfinance", pf_allocation=pf_allocation, start_date=start_date #, end_date=end_date
    )

    #pf.ef_plot_efrontier()
    #pf.properties()

    csv_data['Sharpe'] = 0.0
    csv_data['Volatility'] = 0.0

    for y in tickers:
        y_stock = pf.get_stock(y)
        y_stock_prices = y_stock.data
        v = float(y_stock.volatility)
        sharpe = finquant.quants.sharpe_ratio(float(y_stock.expected_return), v, risk_free_rate=0.005)
        #print(f" {y} : {sharpe} ")

        #csv_data_y = csv_data[csv_data.Ticker==y]
        #csv_data[csv_data.Ticker==y]['sharpe'] = sharpe
        #df.loc[df['age']==3, ['age-group']] = 'toddler'
        #csv_data.loc[(csv_data['Ticker']==y), ['Sharpe']] = sharpe
        csv_data.loc[(csv_data.Ticker == y),'Sharpe']= sharpe
        csv_data.loc[(csv_data.Ticker == y),'Volatility']= v
        csv_data

    return csv_data
    

types = csv_data.Type.unique() #['Crypto','ESG']
#print(types)

map_data_list = []

for type in types:
    print (type)
    csv_data = modify_csv_data_with_map_values(type, csv_data)
    map_data_list.append(csv_data[csv_data.Type==type])

map_data = pd.concat(map_data_list)
map_data.sort_values(['Sharpe','Volatility'],ascending=False).groupby('Type').head(3)
map_data.to_csv('./Resources/map_data.csv')
map_data

Crypto
[*********************100%***********************]  7 of 7 completed

1 Failed download:
- BTTC: No data found, symbol may be delisted
Dividend
[*********************100%***********************]  19 of 19 completed
Tech
[*********************100%***********************]  20 of 20 completed
SP500
[*********************100%***********************]  6 of 6 completed
Bonds
[*********************100%***********************]  20 of 20 completed
Commodities
[*********************100%***********************]  19 of 19 completed
Pharma
[*********************100%***********************]  13 of 13 completed
AI
[*********************100%***********************]  15 of 15 completed
Aggressive
[*********************100%***********************]  7 of 7 completed
